In [1]:
from bn256 import *
import random
from copy import copy
from functools import reduce
import time

In [2]:
def timer(func):
    def _func(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        t = time.clock() - t
        print('{0}: time elapsed {1:.8f} s.' .format(func.__name__, t))
        return res

    return _func

In [3]:
def hamminge_distance(s1, s2):
    l = len(s1)
    assert l == len(s2), "Undefined for sequences of unequal length."
    return sum(el1 != el2 for el1, el2 in zip(s1, s2)) / l

In [4]:
a = (-1, 1)
def gen_vector(size):
    return tuple(random.choice(a) for i in range(size))

In [5]:
v1, v2 = gen_vector(16), gen_vector(16)

In [6]:
print(v1, v2, sep="\n")

(-1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1)
(-1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, -1)


In [7]:
def inverse_G1(curve):
    inv = copy(curve)
    inv.y = inv.y.additive_inverse()
    return inv

In [8]:
def inverse_G2(twist):
    inv = copy(twist)
    inv.negate()
    return inv

In [9]:
def transfer_vec_to_G1(vec):
    return tuple(inverse_G1(curve_G) if p == -1 else curve_G for p in vec)

In [10]:
def transfer_vec_to_G2(vec):
    return tuple(inverse_G2(twist_G) if p == -1 else twist_G for p in vec)

In [11]:
def gen_keys(size):
    return tuple(random.randrange(2, order) for i in range(size))

In [12]:
v1points = transfer_vec_to_G1(v1)
v2points = transfer_vec_to_G2(v2)

In [13]:
v1points

((1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781))

In [14]:
v2points

(((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (20666913350058776956210519119118544732556678129809273996262322366050359951122,17778617556404439934652658462602675281523610326338642107814333856843981424549)),
 ((2116796163654258025501177006657054

In [15]:
h1, h2 = random.randrange(2, order), random.randrange(2, order)
H1, H2 = curve_G.scalar_mul(h1), twist_G.scalar_mul(h2)

In [16]:
h1, h2

(44506407755299420872314292264983248053375090066769998442041817863951314659720,
 4495126981060587054657400517982349121366323176518153311557768172035821745583)

In [17]:
@timer
def master_key_generation():
    s, t = gen_keys(16), gen_keys(16)
    u, v = gen_keys(18), gen_keys(18)
    gen1_h = tuple(point_add(curve_G.scalar_mul(si), H1.scalar_mul(ti)) for si, ti in zip(s, t))
    gen2_h = tuple(point_add(twist_G.scalar_mul(ui), H2.scalar_mul(vi)) for ui, vi in zip(u, v))
    return gen1_h, s, t, gen2_h, u, v

In [18]:
gen1_h, s, t, gen2_h, u, v = master_key_generation()

master_key_generation: time elapsed 19.51244551 s.


In [19]:
gen1_h, s, t, gen2_h, u, v

(((15968089400380036892452143751210297714238194189166032060179725545048425029535, 21370252204629405675662891303853605299074429081070793500608030053577188832981),
  (47492108616463479917865594681761430681831288246040005696423576389813574160981, 52821223281580516350283127147679254373302818757551338033366502251212070965572),
  (31266412766243423114393906411260580808529479476415996310818826877565141361794, 32478426403344300362007440238075982645998078674734276487945849004088155025206),
  (63549575000775814850828290548308557576991976274214164631938490173490421602887, 5446655301530837899355484225187975872388458704381157494197270107487773928966),
  (63096992137624320539217247881402615585344700499775065326790164183282448869608, 21517524018947944718246435764363835753305227247931717608302169128461499216113),
  (58300840958484711185362852387083926854056417204746215889902356922436002153584, 42301981587279220054764853854569815464588702733629684415774967723981302824397),
  (11513899018824074754130650

In [20]:
r0, r1 = random.randrange(2, order), random.randrange(2, order)

In [21]:
@timer
def decryption_key_generation(gen1_h, s, t, c):
    n = len(c)
    
    d01 = curve_G.scalar_mul(r0)
    d02 = H1.scalar_mul(r0)
    
    d03 = c[0].scalar_mul(s[0])
    d04 = c[0].scalar_mul(t[0])
    for i in range(1, n):
        d03 = point_add(d03, c[i].scalar_mul(s[i]))
        d04 = point_add(d04, c[i].scalar_mul(t[i]))
    d03 = point_add(inverse_G1(d03), gen1_h[0].scalar_mul(r0))
    d04 = point_add(inverse_G1(d04), gen1_h[1].scalar_mul(r0))
    
    DK = [d01, d02, d03, d04]
    for i in range(n):
        DK.append(point_add(c[i], gen1_h[i].scalar_mul(r0)))
    
    return tuple(DK)

In [22]:
DK = decryption_key_generation(gen1_h, s, t, v1points)

decryption_key_generation: time elapsed 3.82562397 s.


In [23]:
len(DK)

20

In [24]:
DK

((239548858971116726687506625305014114827466871643497309670505111885952913965, 31756471526535521794777797156943287233023468012056532576996391093488727570596),
 (60885255752173127502381845219640975284210192485416929175433085696847040081329, 12475403650755134496553995397960121620892473905336315487336525597209394891851),
 (61407093883539165625348249329005472946097837032976726988496564517058056487107, 1358722103869902485341183638644738887081651173187371493822283642543834298229),
 (26177267593780814372361607690745050465603396696855096032289490752777657287508, 43587337305171849126596564181224650033632115882465653227878588077224912134605),
 (13100882072851547072698065018260540594849736717565506180723623635033669140283, 6735242011554832367412948010537118325304149091049066371628207659991717980574),
 (59724992644827324715014100982600326764056507249761189252406119142734481257312, 15794003502588104818091175623059155749403286043361988533458047195030778293176),
 (135091721960917664146573966489185534

In [25]:
@timer
def encryption_authentication(gen2_h, u, v, c):
    n = len(c)
    
    c03 = twist_G.scalar_mul(r0)
    c04 = H2.scalar_mul(r0)
    
    CT0 = [c03, c04]
    for i in range(n):
        CT0.append(point_add(c[i], gen2_h[i].scalar_mul(r0)))
        
    c01 = c03.scalar_mul(u[0])
    c02 = c04.scalar_mul(v[0])
    for i in range(1, n + 2):
        c01 = point_add(c01, CT0[i].scalar_mul(u[i]))
        c02 = point_add(c02, CT0[i].scalar_mul(v[i]))
    
    c01 = inverse_G2(c01)
    c02 = inverse_G2(c02)
    
    return (c01, c02) + tuple(CT0)

In [26]:
CT = encryption_authentication(gen2_h, u, v, v2points)

encryption_authentication: time elapsed 14.02705365 s.


In [27]:
len(CT)

20

In [28]:
CT

(((62251553008413875628790343352268866067465466745201287871320600840041652507361,14019424648654944279850993310667122397150986775734827861016180629236086391682), (17132020654131446822940609664011730242751631507643354530080269422124189414376,27818724769737302268356287382466546188650321362783064801584251879716930372924)),
 ((5015651758173782608801627974623015341481178843845173698283729603532025016362,18360022864218162376200819466909360895211376915235138912160152785442734074905), (37584174109445443479580499682920917357549923727159090483338439421210385428058,63627913568873210863582954439535870707143099145501261160107020978379912263193)),
 ((54121639228826601331649688788951672261888362434530669847326641082186814835926,17825756676367880376916022035535241764113820996703572579360497180852513314095), (31491258201405709823860115525047554586004020721911371722721547657170240290451,10394970015506994317841353614497456772613482292452510197354029062018615877510)),
 ((46723741520616639702747191534480661

In [29]:
CT_v1 = encryption_authentication(gen2_h, u, v, transfer_vec_to_G2(v1))
logarithmic_table = [optimal_ate(cki, dki) for cki, dki in zip(CT_v1, DK)]

encryption_authentication: time elapsed 16.00896540 s.


In [30]:
logarithmic_table

[(((20999421645136796396829996079363042660500258961633894508644748627660473986561,6634483953125503109562156277838674634245131162633822947590191458494501261204),(9763976985275705272224510300044557909843725632968172165120719526124906011259,27652200299380445791305045187647890853099578863038893374033604884696022228561),(53219791766865655664688158031795958149664611564112711072657268903682861402938,59099309445667816444151525737853088531871976026124339345017450305278059838712)),((44205392276831221303448435319093322914631484377142787961981838032034264946890,8076654497464726671211395699051687806272511221222463349020326296139336103705),(27745236712645585807427407726193376948163601999146032688490808277823142511675,4188564533702362392041465414769077405890589497504952075090598316818668272491),(39470182703253872607980289418330412709136811459875642419470916057314281559558,63200920261979855593528723600081855774294089706734912617259526810118712432140))),
 (((55052545533414093574766886948371479285398360

In [31]:
@timer
def decryption_authentication(DK, CT):
    return [optimal_ate(cti, dki) for dki, cti in zip(DK, CT)]

In [32]:
DA = decryption_authentication(DK, CT)

decryption_authentication: time elapsed 15.31812902 s.


In [33]:
DA

[(((63346510108847790449200952566705235977474063880084366273911331441493999712186,58662834630876185347724748579548486127349730054811511331485336448660878604988),(23089835715077929299822664563646580118257845182319948382652405279186318622061,58720601098579076563899254983415552246345493453736519211644642299892520170423),(11915613276446905348045486705727103196714619637935666856623303823993836434353,30668252059440571868961808090832202018486402459770607165158341137411362364333)),((49433600493333950637699631714070549641671388530358147591555452807595007940718,22376102402880539935600301423024391454061401803444918876798356041692038110787),(2903703129363571091008945234124900413413225104880814505090577901559258546731,36300594821149366393488007667747479637208534344042544264000503836222218307180),(50626280358293395266676384358627770071681531118349576777449921367918639753622,39307749039432222497783912988407071380469051496717273553781459240612233175664))),
 (((31570556861016317946830234735546507100564

In [34]:
len(logarithmic_table), len(DA)

(20, 20)

In [35]:
hamminge_distance(logarithmic_table[4:], DA[4:])

0.625

In [36]:
hamminge_distance(v1, v2)

0.625